In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comp5423-group-project-squad-20-csv/train-v2.0-no-imposs-q.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from tqdm.auto import tqdm  # for showing progress bar
from datasets import load_dataset


import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizerFast

device = torch.device('cuda:0')
#Using torch by GPU
if torch.cuda.is_available():
    device = torch.device('cuda:0')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device('cpu')

task = "./data/formatted-SQuAD-train.csv"


In [55]:
#Download the dataset from SQuAD
#SQuAD = pd.read_json('./data/train-v2.0.json')
#SQuAD = load_dataset('squad')

SQuAD = pd.read_csv("/kaggle/input/comp5423-group-project-squad-20-csv/train-v2.0-no-imposs-q.csv",dtype=str, keep_default_na = False)
print(f'length of dataset: {len(SQuAD)}')

length of dataset: 86821


In [56]:
# replace all NaN values in 'text' column with 'null'
SQuAD['text'].fillna('null', inplace=True)

# convert 'answer_start' column to integer type
SQuAD['answer_start'] = SQuAD['answer_start'].astype(int)

# create new column 'answer' with desired format
SQuAD['answers'] = SQuAD.apply(lambda row: {'text': row['text'], 'answer_start': row['answer_start']}, axis=1)

# remove original 'text' and 'answer_start' columns
SQuAD.drop(['text', 'answer_start'], axis=1, inplace=True)


{'text': 'null', 'answer_start': 628}

In [57]:
def add_end_idx(answers, contexts):
    new_answers = []
    # loop through each answer-context pair
    for answer, context in tqdm(zip(answers, contexts)):
        # quick reformating to remove lists
        #answer['text'] = answer['text'][0]
        #answer['answer_start'] = answer['answer_start'][0]
        # gold_text refers to the answer we are expecting to find in context
    
        gold_text = str(answer['text'])
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
        new_answers.append(answer)
    return new_answers

In [58]:
def prep_data(dataset):
    questions = dataset['question']
    contexts = dataset['context']
    answers = add_end_idx(
        dataset['answers'],
        contexts
    )
    return {
        'question': questions,
        'context': contexts,
        'answers': answers
    }

In [59]:
#splict the set in train and validate
dataset = prep_data(SQuAD)
#dataset_validation = prep_data(SQuAD['validation'])
print('{:>5,} training samples'.format(len(dataset['question'])))

0it [00:00, ?it/s]

86,821 training samples


In [61]:
#prepare BERT model and tokenizer
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

#print(dataset['answers'][:5])
[str(i) for i in dataset['context'].values]

# tokenize
train = tokenizer([str(i) for i in dataset['context'].values],
                  [str(i) for i in dataset['question'].values],
                  add_special_tokens=True,
                  truncation=True,
                  return_attention_mask=True,  # Construct attn. masks.
                  padding='max_length',
                  return_tensors='pt')

#print(tokenizer.decode(train['input_ids'][0])[:855])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [62]:
def add_token_positions(encodings, answers):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    for i in tqdm(range(len(answers))):

        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length

        shift = 1
        while end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})


add_token_positions(train, dataset['answers'])

  0%|          | 0/86821 [00:00<?, ?it/s]

In [65]:
dataset['answers'][22]

{'text': 'June 2005', 'answer_start': 48, 'answer_end': 57}

In [64]:
#training
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

# build datasets for both our training data
train_dataset = SquadDataset(train)

batch_size = 8

loader = torch.utils.data.DataLoader(train_dataset,
                                     batch_size=8,
                                     shuffle=True)

from transformers import AdamW

model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(4):
    loop = tqdm(loader)
    
    for batch in loop:
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask,
                        start_positions=start_positions,
                        end_positions=end_positions)

        loss = outputs[0]
        loss.backward()
        optim.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

    model.save_pretrained('bert_qa_pt_'+ str(epoch))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/10853 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == "__main__":


KeyboardInterrupt: 